## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Provideniya,RU,64.38,-173.30,44.60,70,90,4.47,overcast clouds
1,1,Bilma,NE,18.69,12.92,89.87,12,0,10.22,clear sky
2,2,Talcahuano,CL,-36.72,-73.12,53.60,87,75,9.17,broken clouds
3,3,Saint-Pierre,RE,-21.34,55.48,68.00,72,4,6.93,clear sky
4,4,Butaritari,KI,3.07,172.79,82.18,75,4,14.34,clear sky


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bilma,NE,18.69,12.92,89.87,12,0,10.22,clear sky
4,4,Butaritari,KI,3.07,172.79,82.18,75,4,14.34,clear sky
10,10,Kapaa,US,22.08,-159.32,84.20,66,1,10.29,clear sky
15,15,Rikitea,PF,-23.12,-134.97,70.25,79,71,21.85,broken clouds
23,23,Say,NE,13.10,2.37,84.20,74,93,9.17,overcast clouds
26,26,Singaraja,ID,-8.11,115.09,85.21,57,99,1.45,overcast clouds
31,31,Airai,TL,-8.93,125.41,76.08,40,100,5.97,overcast clouds
32,32,Guerrero Negro,MX,27.98,-114.06,84.69,39,0,12.50,clear sky
33,33,Atuona,PF,-9.80,-139.03,78.40,75,0,17.94,clear sky
34,34,Tadine,NC,-21.55,167.88,71.31,67,0,19.71,clear sky


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                293
City                   293
Country                293
Lat                    293
Lng                    293
Max Temp               293
Humidity               293
Cloudiness             293
Wind Speed             293
Current Description    293
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df=preferred_cities_df.dropna()

In [22]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bilma,NE,89.87,clear sky,18.69,12.92,
4,Butaritari,KI,82.18,clear sky,3.07,172.79,
10,Kapaa,US,84.20,clear sky,22.08,-159.32,
15,Rikitea,PF,70.25,broken clouds,-23.12,-134.97,
23,Say,NE,84.20,overcast clouds,13.10,2.37,
26,Singaraja,ID,85.21,overcast clouds,-8.11,115.09,
31,Airai,TL,76.08,overcast clouds,-8.93,125.41,
32,Guerrero Negro,MX,84.69,clear sky,27.98,-114.06,
33,Atuona,PF,78.40,clear sky,-9.80,-139.03,
34,Tadine,NC,71.31,clear sky,-21.55,167.88,


In [23]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except(IndexError):
        print("Hotel not found... skipping.")
        
    

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [29]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df=hotel_df.dropna()

In [30]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [49]:
# 11a. Add a marker layer for each city to the map. 

fig = gmaps.figure(center=(25.0, 101.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

AttributeError: 'Figure' object has no attribute 'savefig'